In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from torchvision import transforms

transformation_for_train = transforms.Compose(
    [transforms.Resize((380,380)), 
     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
     ]
)
transformation_for_valntest = transforms.Compose(
    [transforms.Resize((380, 380)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
     ]
)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets

train_path = ".\data"

train_dataset= datasets.ImageFolder(root=train_path, transform=transformation_for_train)
val_dataset= datasets.ImageFolder(root=val_path, transform=transformation_for_valntest)
test_dataset= datasets.ImageFolder(root=test_path, transform=transformation_for_valntest)
batchsize = 32

train_loader= DataLoader(train_dataset, batch_size=batchsize,shuffle=True)
val_loader= DataLoader(val_dataset, batch_size=batchsize,shuffle=True)
test_loader= DataLoader(test_dataset, batch_size=batchsize,shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
dataiter = iter(train_loader)
images, labels = next(dataiter)

def unnormalize(img):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    return img * std + mean


fig, axes = plt.subplots(1, 5, figsize=(15,5))
for i in range(5):
    img = unnormalize(images[i])  
    img = img.permute(1, 2, 0).numpy()  
    img = img.clip(0, 1) 
    axes[i].imshow(img)
    axes[i].axis("off")

plt.show()


In [ ]:
from torchvision import models
import torch.nn as nn 
efficientnetmodel = models.efficientnet_b4(pretrained=True)
no_features = efficientnetmodel.classifier[1].in_features  
efficientnetmodel.classifier[1] = nn.Linear(no_features, 1) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnetmodel = efficientnetmodel.to(device)

In [ ]:
def freeze_everything_except_classifier(model):
    for params in model.features.parameters():
        params.requires_grad=False
    print("phase 1 - only training classifier")

In [ ]:
def unfreeze_last_block(model):
    for name,params in model.named_parameters():
        #if "features.6" in name or "features.7" in name or "classifier" in name:
        if "features.7" in name or "classifier" in name: 
            params.requires_grad = True
        else:
            params.requires_grad = False
    print("phase 2 - training last 2 blocks and classifier")

In [ ]:
def unfreeze_last_two_blocks(model):
    for name, params in model.named_parameters():
        #if "features.4" in name or "features.5" in name or "features.6" in name or "features.7" in name or "classifier" in name:
        if "features.6" in name or "features.7" in name or "classifier" in name:
            params.requires_grad = True
        else:
            params.requires_grad = False
    print("Phase 3 - Training last 4 blocks and classifier")

In [ ]:
def unfreeze_whole_model(model):
    for params in model.parameters():
        params.requires_grad = True 
    print("phase 4 - whole model training")

In [ ]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
#optimiser = optim.Adam(densenetmodel.parameters(),lr=0.001)


In [ ]:
num_epochs = 15
phases = [{"epochs": 5, "unfreeze": freeze_everything_except_classifier, "lr": 0.001},{"epochs": 5, "unfreeze": unfreeze_lasttwo_block, "lr": 0.0001},{"epochs": 5, "unfreeze": unfreeze_last_four_blocks, "lr": 0.00001},{"epochs": 5, "unfreeze": unfreeze_whole_model, "lr": 1e-6}]

for phase_idx, phase in enumerate(phases):
    phase["unfreeze"](efficientnetmodel)
    optimiser = optim.Adam(efficientnetmodel.parameters(), lr=phase["lr"])
    print(f"\n--- starting phase {phase_idx+1}: {phase['unfreeze'].__name__} | Learning Rate: {phase['lr']}")

    for epoch in range(phase["epochs"]):
        efficientnetmodel.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            optimiser.zero_grad()
            outputs = efficientnetmodel(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()

            running_loss += loss.item()
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_accuracy = 100 * correct_train / total_train
        avg_loss = running_loss / len(train_loader)

        print(f"phase {phase_idx+1} | epoch [{epoch+1}/{phase['epochs']}]: "
              f"loss = {avg_loss:.4f}, train acc = {train_accuracy:.2f}%")

        checkpoint_filename = f"efficientnet_phase{phase_idx}_epoch{epoch+1}.pth"
        torch.save(efficientnetmodel.state_dict(), checkpoint_filename)
        print(f"model saved as {checkpoint_filename}")

    efficientnetmodel.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = efficientnetmodel(inputs).squeeze()
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"phase {phase_idx+1} val acc: {val_accuracy:.2f}%\n")
